# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



<div class="alert alert-success">
<b>✅ Комментарий ревьюера: v1 </b>  
    
Если надо посмотреть данные из середины, Метод `sample(5)` позволяет получить данные из серидины таблицы, вытаскивает случайным образом. Параметр - число элементов для отображения</div>


**Вывод**

Есть таблица данных, где имеются столбцы:
children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

Каждая строчка соответсвует поданной заявке на кредит. 

Всего 21525 строчек (соответсвенно обращений).

Данных не хватает в столбцах "days_employed" (общий трудовой стаж) и "total_income" (ежемесячный доход).



## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print('число пропусков столбца "общий трудовой стаж в днях"', len(data[data['days_employed'].isna()]))
print('число пропусков столбца "ежемесячный доход"', len(data[data['total_income'].isna()]))

число пропусков столбца "общий трудовой стаж в днях" 2174
число пропусков столбца "ежемесячный доход" 2174


In [3]:
(2174/21525)*100

10.099883855981417

Пропущено 10% данных  в столбцах "общий трудовой стаж в днях" и "ежемесячный доход". 
Оценим данные в колонке "общий трудовой стаж в днях".

In [4]:
print('пропуски данных в категории безработный:',  'о стаже - ', data[data['income_type'] == 'безработный']['days_employed'].isna().sum(), 'о доходе - ', data[data['income_type'] == 'безработный']['total_income'].isna().sum())
print('пропуски данных в категории в декрете:', 'о стаже - ', data[data['income_type'] == 'в декрете']['days_employed'].isna().sum(), 'о доходе - ', data[data['income_type'] == 'в декрете']['total_income'].isna().sum())
print('пропуски данных в категории госслужащий:','о стаже - ', data[data['income_type'] == 'госслужащий']['days_employed'].isna().sum(), 'о доходе - ', data[data['income_type'] == 'госслужащий']['total_income'].isna().sum())
print('пропуски данных в категории компаньон:', 'о стаже - ', data[data['income_type'] == 'компаньон']['days_employed'].isna().sum(), 'о доходе - ', data[data['income_type'] == 'компаньон']['total_income'].isna().sum())
print('пропуски данных в категории пенсионер:','о стаже - ', data[data['income_type'] == 'пенсионер']['days_employed'].isna().sum(), 'о доходе - ', data[data['income_type'] == 'пенсионер']['total_income'].isna().sum())
print('пропуски данных в категории предприниматель:', 'о стаже - ', data[data['income_type'] == 'предприниматель']['days_employed'].isna().sum(), 'о доходе - ', data[data['income_type'] == 'предприниматель']['total_income'].isna().sum())
print('пропуски данных в категории сотрудник:', 'о стаже - ', data[data['income_type'] == 'сотрудник']['days_employed'].isna().sum(),'о доходе - ', data[data['income_type'] == 'сотрудник']['total_income'].isna().sum())

пропуски данных в категории безработный: о стаже -  0 о доходе -  0
пропуски данных в категории в декрете: о стаже -  0 о доходе -  0
пропуски данных в категории госслужащий: о стаже -  147 о доходе -  147
пропуски данных в категории компаньон: о стаже -  508 о доходе -  508
пропуски данных в категории пенсионер: о стаже -  413 о доходе -  413
пропуски данных в категории предприниматель: о стаже -  1 о доходе -  1
пропуски данных в категории сотрудник: о стаже -  1105 о доходе -  1105


Пропуски данных о стаже и о доходе вероятно находятся у одних и тех же пользователей. 
Категории занятости у который отсутвтсвуют данные - госслужащий, компаньон, пенсионер, предприниматель, сотрудник. Это категории у которых долны быть и доходы и стаж, а значит заменить на нулевые значения нельзя.
Вероятно клиенты не указывали доход и стаж или по ошибке или не хотели их указывать.

Для удобства дальнейшей работы со стажем сделаем дополнительную колонку "стаж в годах", возьмем среднюю продолжительность года в 365 дней.

In [5]:
data['years_employed']=data['days_employed']/365
data.head(1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,-23.116912


Рассмотрим статистику данных о стаже.

In [6]:
years_employed_min= data['years_employed'].min()
years_employed_mean= data['years_employed'].mean()
years_employed_max = data['years_employed'].max()
years_employed_median = data['years_employed'].median()
print('минимальный стаж', years_employed_min, 'лет')
print('максимальный стаж', years_employed_mean, 'лет')
print('средний стаж', years_employed_max, 'лет')
print('медиана стажа', years_employed_median, 'лет')

минимальный стаж -50.38068465909146 лет
максимальный стаж 172.73013057937914 лет
средний стаж 1100.6997273296713 лет
медиана стажа -3.296902818549285 лет


Обращает на себя внимание 2 факта:
1) отрицательное значение минимального стажа.
2) средний стаж 172 года (вроятно из-за того что кто-то указал в стаже нереальное количество дней и поэтому максимальный стаж оказался 1100 лет). Просто заменить пропуски на среднее нельзя.
Сначала разберемся с первой проблемой. Переведем значения столбца "общий трудовой стаж в днях" в абсолютные числа.

In [7]:
data['days_employed'] = data['days_employed'].abs()
data['years_employed']=data['years_employed'].abs()
years_employed_min= data['years_employed'].min()
years_employed_mean= data['years_employed'].mean()
years_employed_max = data['years_employed'].max()
years_employed_median = data['years_employed'].median()
print('минимальный стаж', years_employed_min, 'лет')
print('максимальный стаж', years_employed_mean, 'лет')
print('средний стаж', years_employed_max, 'лет')
print('медиана стажа', years_employed_median, 'лет')

минимальный стаж 0.06614146093282515 лет
максимальный стаж 183.32802440225305 лет
средний стаж 1100.6997273296713 лет
медиана стажа 6.0115631969279315 лет


Отрицательных значений нет, теперь минимальный стаж 0 дней.

Определим какой процент данных имеет стаж более 70 лет.

In [8]:
years_employed_unreal = data[data['years_employed']>70]['years_employed']
print((years_employed_unreal.count()/data['years_employed'].count())*100)


17.80269753501111


17% клиентов с завышенным стажем. Просто удалить эти данные нельзя, заменим их на более реальные 70 лет.

In [9]:
yaers_data_grouped = data.groupby('income_type').agg({'years_employed': ['median', 'mean', 'max','min', 'count']})
print(yaers_data_grouped)

                years_employed                                             
                        median         mean          max         min  count
income_type                                                                
безработный        1003.873021  1003.873021  1083.021476  924.724567      2
в декрете             9.032219     9.032219     9.032219    9.032219      1
госслужащий           7.368132     9.314786    41.624746    0.109463   1312
компаньон             4.239403     5.784998    48.261817    0.082727   4577
пенсионер          1000.584401  1000.009565  1100.699727  900.626632   3443
предприниматель       1.426981     1.426981     1.426981    1.426981      1
сотрудник             4.312884     6.373970    50.380685    0.066141  10014
студент               1.585621     1.585621     1.585621    1.585621      1


В среднем по 1000 лет в стаже указали безработные и пенсионеры. 

In [10]:
days_employed_grouped = data.groupby('income_type').agg({'days_employed': ['median', 'mean', 'max','min', 'count']})
print(days_employed_grouped)

                 days_employed                                               \
                        median           mean            max            min   
income_type                                                                   
безработный      366413.652744  366413.652744  395302.838654  337524.466835   
в декрете          3296.759962    3296.759962    3296.759962    3296.759962   
госслужащий        2689.368353    3399.896902   15193.032201      39.954170   
компаньон          1547.382223    2111.524398   17615.563266      30.195337   
пенсионер        365213.306266  365003.491245  401755.400475  328728.720605   
предприниматель     520.848083     520.848083     520.848083     520.848083   
сотрудник          1574.202821    2326.499216   18388.949901      24.141633   
студент             578.751554     578.751554     578.751554     578.751554   

                        
                 count  
income_type             
безработный          2  
в декрете            1  
госсл

    Заменим безработным  стаж на более реальный 0 лет

Безработные клиенты при заполнении анкет указали невозможное количество дней стажа 395302.838654 и 337524.466835 (около 1000 лет) величина завышена примерно в 100 раз. помимо очень большой величины обращает внимание тот факт что это дробная величина с точностью до 6 знака после запятой. Вероятно это техническая ошибка при заполнении анкеты или при формировании таблицы. 
Уменьшим безработных в 100 раз.

In [11]:
data.loc[data['income_type'] == 'безработный', 'days_employed'] = data['days_employed']/100
data.loc[data['income_type'] == 'безработный', 'years_employed'] = data['years_employed']/100
days_data_grouped = data.groupby('income_type').agg({'days_employed': ['median', 'mean', 'max','min', 'count']})
yaers_data_grouped = data.groupby('income_type').agg({'years_employed': ['median', 'mean', 'max','min', 'count']})
print(yaers_data_grouped)
print(days_data_grouped) 

                years_employed                                             
                        median         mean          max         min  count
income_type                                                                
безработный          10.038730    10.038730    10.830215    9.247246      2
в декрете             9.032219     9.032219     9.032219    9.032219      1
госслужащий           7.368132     9.314786    41.624746    0.109463   1312
компаньон             4.239403     5.784998    48.261817    0.082727   4577
пенсионер          1000.584401  1000.009565  1100.699727  900.626632   3443
предприниматель       1.426981     1.426981     1.426981    1.426981      1
сотрудник             4.312884     6.373970    50.380685    0.066141  10014
студент               1.585621     1.585621     1.585621    1.585621      1
                 days_employed                                               \
                        median           mean            max            min   
income

Уменьшили ошибочные завышенные стажи в стаж в днях и годах в категории безработных в 100 раз.

Заменим пенсионерам со стажем более 50 лет, стаж на более реальные 50 лет (так как банки обычно ограничивают возраст заемщика 70 лет 70-20=50).

In [12]:
data.loc[data['days_employed'] > 365*50, 'days_employed'] = 365*50
data.loc[data['years_employed'] > 50, 'years_employed'] = 50
days_data_grouped = data.groupby('income_type').agg({'days_employed': ['median', 'mean', 'max','min', 'count']})
yaers_data_grouped = data.groupby('income_type').agg({'years_employed': ['median', 'mean', 'max','min', 'count']})
print(yaers_data_grouped)
print(days_data_grouped)

                years_employed                                        
                        median       mean        max        min  count
income_type                                                           
безработный          10.038730  10.038730  10.830215   9.247246      2
в декрете             9.032219   9.032219   9.032219   9.032219      1
госслужащий           7.368132   9.314786  41.624746   0.109463   1312
компаньон             4.239403   5.784998  48.261817   0.082727   4577
пенсионер            50.000000  50.000000  50.000000  50.000000   3443
предприниматель       1.426981   1.426981   1.426981   1.426981      1
сотрудник             4.312884   6.373932  50.000000   0.066141  10014
студент               1.585621   1.585621   1.585621   1.585621      1
                days_employed                                                 
                       median          mean           max           min  count
income_type                                                  

In [13]:
days_employed_mean = data['days_employed'].mean()
days_employed_max = data['days_employed'].max()
days_employed_median = data['days_employed'].median()
days_employed_min = data['days_employed'].min()
print('минимальный стаж', days_employed_min/365, 'лет')
print('максимальный стаж', days_employed_max/365, 'лет')
print('средний стаж', days_employed_mean/365, 'лет')
print('медиана стажа', days_employed_median/365, 'лет')


минимальный стаж 0.06614146093282515 лет
максимальный стаж 50.0 лет
средний стаж 14.196145855975509 лет
медиана стажа 6.0115631969279315 лет


Заменим пропуски данных о стаже на среднюю величину.

In [14]:
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data['years_employed'] = data['years_employed'].fillna(data['years_employed'].mean())
data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
years_employed         0
dtype: int64

In [15]:
print('пропуски данных о доходе в категории безработный', data[data['income_type'] == 'безработный']['total_income'].isna().sum())
print('пропуски данных о доходе в категории в декрете', data[data['income_type'] == 'в декрете']['total_income'].isna().sum())
print('пропуски данных о доходе в категории госслужащий',data[data['income_type'] == 'госслужащий']['total_income'].isna().sum())
print('пропуски данных о доходе в категории компаньон',data[data['income_type'] == 'компаньон']['total_income'].isna().sum())
print('пропуски данных о доходе в категории пенсионер',data[data['income_type'] == 'пенсионер']['total_income'].isna().sum())
print('пропуски данных о доходе в категории предприниматель',data[data['income_type'] == 'предприниматель']['total_income'].isna().sum())
print('пропуски данных о доходе в категории сотрудник',data[data['income_type'] == 'сотрудник']['total_income'].isna().sum())

пропуски данных о доходе в категории безработный 0
пропуски данных о доходе в категории в декрете 0
пропуски данных о доходе в категории госслужащий 147
пропуски данных о доходе в категории компаньон 508
пропуски данных о доходе в категории пенсионер 413
пропуски данных о доходе в категории предприниматель 1
пропуски данных о доходе в категории сотрудник 1105


пропуски данных о зарплате есть в категориях госслужащий, компаньон, пенсионер, предприниматель, сотрудник.

сгруппируем данные о зарплате по типу занятости

In [16]:
salary_data_grouped = data.groupby('income_type').agg({'total_income': ['median', 'mean', 'max','min', 'count']})
print(salary_data_grouped)

                  total_income                                              \
                        median           mean           max            min   
income_type                                                                  
безработный      131339.751676  131339.751676  2.027225e+05   59956.991984   
в декрете         53829.130729   53829.130729  5.382913e+04   53829.130729   
госслужащий      150447.935283  170898.309923  9.104515e+05   29200.077193   
компаньон        172357.950966  202417.461462  2.265604e+06   28702.812889   
пенсионер        118514.486412  137127.465690  7.351033e+05   20667.263793   
предприниматель  499163.144947  499163.144947  4.991631e+05  499163.144947   
сотрудник        142594.396847  161380.260488  1.726276e+06   21367.648356   
студент           98201.625314   98201.625314  9.820163e+04   98201.625314   

                        
                 count  
income_type             
безработный          2  
в декрете            1  
госслужащий     

Средняя зарплата по категориям различается на порядок.
Пропуски данных по зарплате заменим на средние значения в категории.

Сначала введем переменные среднего дохода для отдельных категорий, имеющих пропуски данных о доходе.

In [17]:
state_income = data[data['income_type'] == 'госслужащий']['total_income'].median()
companion_income = data[data['income_type'] == 'компаньон']['total_income'].median()
pensioner_income = data[data['income_type'] == 'пенсионер']['total_income'].median()
businessman_income = data[data['income_type'] == 'предприниматель']['total_income'].median()
worker_income = data[data['income_type'] == 'сотрудник']['total_income'].median()
student_income = data[data['income_type'] == 'студент']['total_income'].median()

print('средний доход у госслужащих', state_income)
print('средний доход у компаньонов', companion_income)
print('средний доход у пенсионеров', pensioner_income)
print('средний доход у предпринимателей', businessman_income)
print('средний доход у сотрудников', worker_income)
print('средний доход у студентов', student_income)



средний доход у госслужащих 150447.9352830068
средний доход у компаньонов 172357.95096577113
средний доход у пенсионеров 118514.48641164352
средний доход у предпринимателей 499163.1449470857
средний доход у сотрудников 142594.39684740017
средний доход у студентов 98201.62531401133


Заменим пропуски данных о зарплате медианами по категории:

In [18]:
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

In [19]:
salary_data_grouped = data.groupby('income_type').agg({'total_income': ['median', 'mean', 'max','min', 'count']})
print(salary_data_grouped)

                  total_income                                              \
                        median           mean           max            min   
income_type                                                                  
безработный      131339.751676  131339.751676  2.027225e+05   59956.991984   
в декрете         53829.130729   53829.130729  5.382913e+04   53829.130729   
госслужащий      150447.935283  168837.854082  9.104515e+05   29200.077193   
компаньон        172357.950966  199414.466116  2.265604e+06   28702.812889   
пенсионер        118514.486412  135133.907484  7.351033e+05   20667.263793   
предприниматель  499163.144947  499163.144947  4.991631e+05  499163.144947   
сотрудник        142594.396847  159513.331868  1.726276e+06   21367.648356   
студент           98201.625314   98201.625314  9.820163e+04   98201.625314   

                        
                 count  
income_type             
безработный          2  
в декрете            1  
госслужащий     

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
years_employed      21525 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


In [21]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,18250.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,50.000000
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


**Вывод**


Пропущенные данные были обнаружены в строках "общий трудовой стаж в днях" и "ежемесячный доход". При этом пропуски коррелируют между собой, и если у клиента отсутствовала информация о стаже, то отсутствовала и о доходе. Пропуски данных были у в категориях госслужащий, компаньон, пенсионер, предприниматель, сотрудник, это категории занятости при которых оба значения должны были бы быть. Возможно клиенты сознательно не указывали эти данные потому что скрывали или пропуски получились в результате технической ошибки при формировании таблицы.
Пропуски в данных о трудовом стаже заменили на среднее значение трудового стажа. Для корректного отображения среднего трудового стажа перевели отрицательные значения стажа в абсолютные и заменили завышенные значения трудового стажа. Так у некоторых клиентов величина была завышена до 1000 лет. Помимо очень большой величины обращает внимание тот факт что это дробная величина с точностью до 6 знака после запятой. Вероятно это техническая ошибка при заполнении анкеты или при формировании таблицы. Уменьшили стаж безработных клиентов в 100 раз, а клиентам-пенсионерам до 50 лет.
Пропуски данных о ежемесячных доходах заменили на средние значения по катерогиям типа занятости.

### Замена типа данных

У столбца "days_employed" (общий трудовой стаж в днях) тип данных float64, тогда так логичнее что бы это были целочисленные данные.
Вещественные данные в целочисленные переводят методом astype().

In [22]:
data['days_employed'] = data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
years_employed      21525 non-null float64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


**Вывод**


заменили тип данных в столбце "days_employed" (общий трудовой стаж в днях) на целочисленные (int64).

### Обработка дубликатов

Посчитаем количество дубликатов в таблице методом value_counts().

In [23]:
data.duplicated().value_counts()

False    21471
True        54
dtype: int64

In [24]:
54/21471*100

0.25150202598854265

Дублирующих значений 0,25%, такое небольшое количество дубликатов могло быть следствием полностью случайной ошибки, например при заполнении таблицы сотрудник отвлекся и записал в базу данных одного и того же клиента дважды.
Эти данные не внесут существенного вклада в дальнейшее исследование, их можно удалить методом drop_duplicates().
После чего проверим число дубликатов методом duplicated().

In [25]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**


Удалили дубликаты данных.

### Лемматизация

Импортируем библиотеку pymystem3

In [26]:
from pymystem3 import Mystem
m = Mystem()

Лемматизируем слова в значениях столбца с целями получения кредита и посчитаем сколько раз встречается каждое слово.
Выведем слова и частоту на экран.

In [27]:
text =  ' '.join(data['purpose'])
lemmas = m.lemmatize(text)
from collections import Counter
print(Counter(lemmas))

Counter({' ': 55066, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'подержать': 853, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [28]:
data['purpose_lemmas'] = data['purpose'].apply(m.lemmatize)
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0          18250         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  years_employed  \
0               покупка жилья       23.116912   
1     приобретение автом

**Вывод**


Чаще всего кредит берут для покупки недвижимости (жилой и коммерческой), покупки автомобиля, оплаты образования, проведения свадьбы и ремонта.

### Категоризация данных

Уточним сколько детей встречается у клиентов банка.

In [29]:
 data['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке с количеством детей есть данные с отричательным занчением. Переведем их в вещественные.

In [30]:
data['children'] = data['children'].abs()

Так же есть клинты указавшие 20 детей, посмотрим в какой возрастной категирии они находятся.

In [31]:
children_data_grouped = data.groupby('dob_years').agg({'children': [ 'max', 'count']})
print(children_data_grouped)

          children      
               max count
dob_years               
0               20   101
19               0    14
20               2    51
21              20   111
22               2   183
23              20   253
24              20   264
25              20   357
26              20   408
27              20   493
28               4   503
29              20   544
30              20   538
31              20   559
32              20   509
33              20   581
34              20   601
35              20   616
36              20   554
37              20   536
38              20   597
39              20   572
40              20   607
41              20   606
42              20   596
43              20   512
44              20   545
45              20   497
46              20   473
47               3   477
48              20   537
49              20   508
50              20   513
51              20   448
52              20   484
53              20   459
54              20   476


По 20 детей указали пользователи всех возрастных групп (с возрастами 0, 21, 23-27, 29-46, 48-57, 59-63, 64, 69), в в целом это нереальное число детей для любой возрастной группы.
проверим сколько процентов пользователей с таким числом детей.

In [32]:
73/21525*100

0.33914053426248547

0.3% данных, величина которой можно принебречь и не выяснять причины по которым появилась ошибка. Удалим данные о пользователях с завышенным количеством детей.

In [33]:
data['children'] = data['children'].loc[data['children'] < 20]
data['children'].value_counts()

0.0    14107
1.0     4856
2.0     2052
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

Сгруппируем данные по количеству детей.

Создадим таблицу data_children в которую перенесем столбцы 'children' и 'debt'.

In [34]:
data_children = data[['children', 'debt']]
data_children.head()

,children,debt
0,1.0,0
1,1.0,0
2,0.0,0
3,3.0,0
4,0.0,0


Запишем правила классификации клиентов как функции. На вход функции попадает количество детей, а возвращает она категорию клиента.

In [35]:
def children_group(childrens):
    if childrens< 1:
        return 'без детей'
    if childrens<=2 or childrens>=4 or childrens>=5:
        return '1-2 детей'
    return 'многодетный'

Создадим отдельный столбец с категориями количества детей, и в его ячейках запишем значения, возвращаемые функцией.
Для этого нужен метод apply()

In [36]:
data_children['children_group'] = data_children['children'].apply(children_group)
data_children.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,children,debt,children_group
0,1.0,0,1-2 детей
1,1.0,0,1-2 детей
2,0.0,0,без детей
3,3.0,0,многодетный
4,0.0,0,без детей
5,0.0,0,без детей
6,0.0,0,без детей
7,0.0,0,без детей
8,2.0,0,1-2 детей
9,0.0,0,без детей


В столбце family_status значится семейный статус клиентов, в столбце family_status_id — кодовое числовое значание семейного статусаи, а просрочки платежа — целые числа в столбце debt. Количество клиентов каждого семеного статуса можно посчитать методом value_counts(). Посчитаем количества клиентов из каждой группы семейного статуса и их кодового обозначения, в том числе что бы понять что значат эти кодовые обозначения.

In [37]:
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [38]:
data['family_status_id'].value_counts()

0    12344
1     4163
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Таким образом получаются следующие значения кода семейного статуса:
0 - женат / замужем;
1 - гражданский брак;
2 - вдовец / вдова;
3 - в разводе;
4 - не женат / не замужем.
Количество клиентов в каждой категории находятся в пределах одного порядка, объединять группы тоже нет смысла, так как психология и поведение у данных групп может быть разным.
'family_status_id' может выступать в роли колонки категоризации, так как в ней зашифрована категория семейного положения.

Создадим таблицу data_family в которую перенесем столбцы 'family_status', 'family_status_id' и 'debt'.

In [39]:
data_family = data[['family_status', 'family_status_id', 'debt']]
data_family.head()

,family_status,family_status_id,debt
0,женат / замужем,0,0
1,женат / замужем,0,0
2,женат / замужем,0,0
3,женат / замужем,0,0
4,гражданский брак,1,0


Познакомимся с данными о зарплатах зарплат у клиентов.

In [40]:
data['total_income'].value_counts().sort_values()


141825.901103       1
162416.137763       1
122140.327609       1
175780.973093       1
128056.263801       1
                 ... 
499163.144947       2
150447.935283     145
118514.486412     395
172357.950966     504
142594.396847    1077
Name: total_income, Length: 19353, dtype: int64

In [41]:
data['total_income'].min()

20667.26379327158

In [42]:
data['total_income'].max()

2265604.028722744

In [43]:
data['total_income'].median()

142594.39684740017

Исходя из статистических данных можно выделить 5 групп клиентов по доходу:
низкий - менее 70 тыс.
ниже среднего - от 70 до 100 тыс.
средний - от 100 до 140 тыс.
выше среднего - от 140 до 200 тыс.
высокий - от 200 до 300 тыс.
сверхвысокий - более 300 тыс.

Проверим количество клиентов в каждой из групп.

In [44]:
data[data['total_income']<70000]['total_income'].count()

1474

In [45]:
data[(data['total_income']>=70000) & (data['total_income']<100000)]['total_income'].count()


2989

In [46]:
data[(data['total_income']>=100000) & (data['total_income']<140000)]['total_income'].count()

5036

In [47]:
data[(data['total_income']>=140000) & (data['total_income']<200000)]['total_income'].count()

6905

In [48]:
data[(data['total_income']>=200000) & (data['total_income']<300000)]['total_income'].count()

3584

In [49]:
data[(data['total_income']>=300000)]['total_income'].count()

1483

Создадим таблицу data_income в которую перенесем столбцы 'total_income' и 'debt'.

In [50]:
data_income = data[['total_income', 'debt']]
data_income.head()

,total_income,debt
0,253875.639453,0
1,112080.014102,0
2,145885.952297,0
3,267628.550329,0
4,158616.077870,0


Таким образом количество клиетов в группе с низким доходом приблизительно равно количеству с сверхвысоким, количество клиентов с высоким доходом и ниже среднего так же при грубой оценке равно, как и количество клиентов со средним и выше среднего доходами. такое количественное распределение по группам нас устраивает.

Запишем правила классификации клиентов по доходу как функцию. На вход функции попадает средний доход за месяц, а возвращает она категорию клиента.

In [51]:
def income_group(income):
    if income < 70000:
        return 'низкий'
    if (income >=70000) & (income < 100000):
        return 'ниже среднего'
    if (income >=100000) & (income < 140000):
        return 'средний'
    if (income >=140000) & (income < 200000):
        return 'выше среднего'
    if (income >=20000) & (income < 300000):
        return 'высокий'
    return 'сверхвысокий'

Создадим отдельный столбец с категориями дохода, и в его ячейках запишем значения, возвращаемые функцией. Для этого нужен метод apply()

In [52]:
data_income['income_group'] = data_income['total_income'].apply(income_group)
data_income.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,total_income,debt,income_group
0,253875.639453,0,высокий
1,112080.014102,0,средний
2,145885.952297,0,выше среднего
3,267628.550329,0,высокий
4,158616.077870,0,выше среднего
5,255763.565419,0,высокий
6,240525.971920,0,высокий
7,135823.934197,0,средний
8,95856.832424,0,ниже среднего
9,144425.938277,0,выше среднего


Для ответа на вопрос "Как разные цели кредита влияют на его возврат в срок?" нужно выделить категории целей клиентов.
Для этого создадим таблицу со столбцами цель кредита, лендинг целей и задолженности.

In [53]:
data_purpose = data[['purpose', 'purpose_lemmas', 'debt']]
print(data_purpose.head())

                      purpose                        purpose_lemmas  debt
0               покупка жилья               [покупка,  , жилье, \n]     0
1     приобретение автомобиля     [приобретение,  , автомобиль, \n]     0
2               покупка жилья               [покупка,  , жилье, \n]     0
3  дополнительное образование  [дополнительный,  , образование, \n]     0
4             сыграть свадьбу             [сыграть,  , свадьба, \n]     0


Чаще всего кредит берут для покупки недвижимости (жилой и коммерческой), покупки автомобиля, оплаты образования, проведения свадьбы и ремонта. выделим 6 категорий цели кредита - жилье(леммы жилой и жилье), образование (лемма образование), коммерческий (лемма коммерческий), свадьба (лемма свадьба), автомобиль (лемма автомобиль) и другое (если ни одна из лемм не найдена).

Запишем правила классификации клиентов по цели как функцию. На вход функции попадает лемма цели кредита, а возвращает она категорию клиента.

In [54]:
def purpose_group(purpose_lemmas):
    for value in purpose_lemmas:
        if 'жилье' in value or 'жилой' in value:
            return 'жилье'
        if 'автомобиль' in value:
            return 'автомобиль'
        if 'образование' in value:
            return 'образование'
        if 'коммерческий' in value:
            return 'коммерческий'
        if 'свадьба' in value:
            return 'свадьба'
    return 'другое'

In [55]:
data_purpose['purpose_group'] = data_purpose['purpose_lemmas'].apply(purpose_group)
print(data_purpose.head(10)) 

                      purpose                             purpose_lemmas  \
0               покупка жилья                    [покупка,  , жилье, \n]   
1     приобретение автомобиля          [приобретение,  , автомобиль, \n]   
2               покупка жилья                    [покупка,  , жилье, \n]   
3  дополнительное образование       [дополнительный,  , образование, \n]   
4             сыграть свадьбу                  [сыграть,  , свадьба, \n]   
5               покупка жилья                    [покупка,  , жилье, \n]   
6           операции с жильем             [операция,  , с,  , жилье, \n]   
7                 образование                          [образование, \n]   
8       на проведение свадьбы        [на,  , проведение,  , свадьба, \n]   
9     покупка жилья для семьи  [покупка,  , жилье,  , для,  , семья, \n]   

   debt purpose_group  
0     0         жилье  
1     0    автомобиль  
2     0         жилье  
3     0   образование  
4     0       свадьба  
5     0         жил

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


**Вывод**

Выделили следующие категории:
По количеству детей - без детей, 1-2 детей, много детные.
По доходу - низкий (меньше 70 тыс), ниже среднего (от 70 до 100 тыс), средний (от 100 до 140 тыс), выше среднего (от 140 до 200тыс), высокий (от 200 тыс до 300 тыс) и сверхвысокий (больше 300 тыс).
По цели кредита - жилье, автомобиль, свадьба, коммерческий и другое.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Таблица с данными о детях и задолженностях

In [56]:
data_children.head()

,children,debt,children_group
0,1.0,0,1-2 детей
1,1.0,0,1-2 детей
2,0.0,0,без детей
3,3.0,0,многодетный
4,0.0,0,без детей


Вычислим процент задолженностей по кредиту в категориях по количеству детей, для этого вычислим число невозвратов и поделим на общее количество клиентов и умножим на 100.

In [57]:
data_children_debt = data_children.groupby('children_group')['debt'].sum()/data_children.groupby('children_group')['debt'].count()*100
data_children_debt.sort_values()

children_group
без детей      7.535266
многодетный    8.620690
1-2 детей      9.241161
Name: debt, dtype: float64

**Вывод**


Больше всего невозратов в категории клиентов с 1-2 детьми , клиенты без детей имеют меньше всего невозвратов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Таблица с данными о семейном положении и задолженностях

In [58]:
data_family.head()

,family_status,family_status_id,debt
0,женат / замужем,0,0
1,женат / замужем,0,0
2,женат / замужем,0,0
3,женат / замужем,0,0
4,гражданский брак,1,0


Вычислим процент задолженностей по кредиту в категориях разного семейного статуса, для этого вычислим число невозвратов и поделим на общее количество клиентов и умножим на 100.

In [59]:
data_family_debt = data_family.groupby('family_status')['debt'].sum()/data_family.groupby('family_status')['debt'].count()*100
data_family_debt.sort_values()

family_status
вдовец / вдова           6.569343
в разводе                7.112971
женат / замужем          7.542126
гражданский брак         9.320202
Не женат / не замужем    9.750890
Name: debt, dtype: float64

**Вывод**

Больше всего невозвратов в категориях "Не женат / не замужем" и "гражданский брак".
Меньше всего невозратов в категории "вдовец / вдова".

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Таблица с данными о уровне дохода и задолженностях

In [60]:
data_income.head()

,total_income,debt,income_group
0,253875.639453,0,высокий
1,112080.014102,0,средний
2,145885.952297,0,выше среднего
3,267628.550329,0,высокий
4,158616.077870,0,выше среднего


Вычислим процент задолженностей по кредиту в категориях разного дохода, для этого вычислим число невозвратов и поделим на общее количество клиентов и умножим на 100.

In [61]:
data_income_debt = data_income.groupby('income_group')['debt'].sum()/data_income.groupby('income_group')['debt'].count()*100
data_income_debt.sort_values()

income_group
низкий           6.852103
высокий          7.031250
сверхвысокий     7.147674
ниже среднего    8.464369
средний          8.478952
выше среднего    8.718320
Name: debt, dtype: float64

**Вывод**

Меньше всего невозратов в категории низкого дохода.
Больше всего невозратов у клиентов со средними доходами (ниже среднего, средний и выше среднего)

- Как разные цели кредита влияют на его возврат в срок?

Таблица с данными о целях кредита и задолженностях

In [62]:
data_purpose.head()

,purpose,purpose_lemmas,debt,purpose_group
0,покупка жилья,"[покупка, , жилье, \n]",0,жилье
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",0,автомобиль
2,покупка жилья,"[покупка, , жилье, \n]",0,жилье
3,дополнительное образование,"[дополнительный, , образование, \n]",0,образование
4,сыграть свадьбу,"[сыграть, , свадьба, \n]",0,свадьба


In [63]:
data_purpose_debt = data_purpose.groupby('purpose_group')['debt'].sum()/data_purpose.groupby('purpose_group')['debt'].count()*100
data_purpose_debt.sort_values()

purpose_group
жилье           6.974701
другое          7.506562
коммерческий    7.545732
свадьба         7.965739
образование     9.217738
автомобиль      9.354689
Name: debt, dtype: float64

**Вывод**

Меньше всего задолженностей у клиентов с целью покупки жилой недвижимости.
Больше всего задолженностей у клиентов с кредитом на автомобиль и образование.

## Шаг 4. Общий вывод

Семейное положение,количество детей клиента, уровень дохода и цель кредита влияют на факт погашения кредита в срок.
В целом процент задолженности по кредиту в группах колеблется от 7% до 9%.
Так клиенты без детей, состоявшие в законных отношениях (не столь важно состоящие на данный момент или хотя бы имевшие факт законных отношений), с целью кредита на приобретение жилья и как ни странно с низким уровнем дохода имеют меньший процент невозрата в своих разрядах.
Так клиенты с 1-2 детьми, не состоявшие в законных отношениях , с целью кредита на приобретение автомобиля или обучение и  со средним уровнем дохода имеют больший процент невозрата в своих разрядах.